In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON"); // кнопка начала записи
var t = document.createTextNode("Press to start recording"); // текст кнопки начала записи

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

// функция создания потока аудио
var handleSuccess = function(stream){
  gumStream = stream;
  var options = {
    //bitsPerSeconds: 8000, //хром игнорирует, всегда 48к
    mimeType: 'audio/webm;codecs=opus'
    //mimeType: 'audio/webm;codecs=pcm'
  };

  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    };
  };
  recorder.start(); // начинаем запись
};

recordButton.innerText = "Recording... press to stop"; // меняем текст кнопки

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

// включение\выключение записи
function toggleRecording() {
  if (recorder && recorder.state == "recording") {
    recorder.stop(); // останавливаем запись
    gumStream.getAudioTracks()[0].stop();
    recordButton.innerText = "Saving the recording... pls wait!"; // меняем текст кнопки
  }
}

// пауза
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

// эти данные будут отправляться в Python
var data = new Promise(function(resolve) {
  recordButton.onclick = function() {
    toggleRecording();
    sleep(2000).then(function() {
      // ждем 2000ms доступности данных...
      resolve(base64data.toString());
    });
  };
});
</script>
"""

# функция получения записи
def get_audio():
  display(HTML(AUDIO_HTML)) # создаем HTML
  data = eval_js("data") # выполняем
  binary = b64decode(data.split(",")[1]) # переводим в байтовый вид
  # обработка
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  ) 
  output, err = process.communicate(input=binary)
  # байты надо расставить в правильной порядке для Wav
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
    q, r = divmod(q, 256)
    b.append(r)
  
  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  # читаем в Wav из буфера
  sr, audio = wav_read(io.BytesIO(riff))
  # sr - частота дискретизации
  # audio - звук в формате Wav.
  return audio, sr

audio, sr = get_audio()